In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
import xgboost as xgb
import pandas as pd
from features.UserJoin import UserJoin
from features.UserJoin import submit, diff, plt_month, plt_day, load_ids, diff_set
import config as C
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.metrics import f1_score
from pathlib import Path

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# data
ds = UserJoin()

In [3]:
# def check_false(target):
#     for i in Path('submit_csv').glob('base*.csv'):
#         cmp = load_ids(i.name)
#         print(i.name, target - cmp, len(target & cmp))

def infot(target):
    for i in Path('submit_csv').glob('3_10_base*.csv'):
        cmp = load_ids(i.name)
        print(i.name, target - cmp, len(target & cmp))

def info(f):
    print(f)
    target = load_ids(f)
    infot(target)

In [5]:
# t = []
# fs = list(Path('submit_csv').glob('base*.csv'))
# for i, f1 in enumerate(fs):
#     for j in range(i+1, len(fs)):
#         f2 = fs[j]
#         s1 = load_ids(f1.name)
#         s2 = load_ids(f2.name)
#         if len(s1) > 100 or len(s2) > 100 or ('68' in f1.name) or ('68' in f2.name):
#             continue
#         t.append(i)
#         t.append(j)
#         print(f1.name, f2.name, len(s1), len(s2), len(s1|s2), len(s1&s2))


In [18]:
known = pd.read_csv('submit_csv/label.csv')
ids_known = set(known.id.unique())

ids = [load_ids(i) for i in ['3_10_base_32_24.csv', '3_10_base_32_27.csv', '3_10_base_34_27.csv', '3_10_base_35_27.csv',]]
from functools import reduce

inner = reduce(lambda x,y: x&y, ids)
all = reduce(lambda x,y: x|y, ids)

target_inner = inner - ids_known
target_all = all - ids_known
print(len(target_inner), len(target_all))


13 32


In [215]:
def checkcv(ids, col='pq_z'):
    if col in ds.month.columns:
        t = ds.month
    else:
        t = ds.day
    t = t[t[col] != 0]
    t = t.drop(columns=['id'])
    tmpcv = t[t.index.isin(ids)].groupby('id').std()[[col]] / t[t.index.isin(ids)].groupby('id').mean()[[col]]
    t2 = t[t.index.isin(C.minerids)].groupby('id').std()[[col]] / t[t.index.isin(C.minerids)].groupby('id').mean()[[col]]
    tmpcv.hist(bins=30)
    plt.title('test')
    t2.hist(bins=10)
    plt.title('mineer')
    plt.show()
    return tmpcv, t2

def checkcv_day_holiday(ids, col='kwh'):
    if col in ds.month.columns:
        t = ds.month
    else:
        t = ds.day
    t = t.drop(columns=['id'])


    g1 = t[(t.index.isin(ids)) & (t.type == 0)].groupby('id')
    tmpcv = g1.std()[[col]] / g1.mean()[[col]]
    g2 = t[(t.index.isin(C.minerids)) & (t.type == 0)].groupby('id')
    tcv = g2.std()[[col]] / g2.mean()[[col]]

    g1 = t[(t.index.isin(ids)) & (t.type != 0)].groupby('id')
    tmpcv2 = g1.std()[[col]] / g1.mean()[[col]]
    g2 = t[(t.index.isin(C.minerids)) & (t.type != 0)].groupby('id')
    tcv2 = g2.std()[[col]] / g2.mean()[[col]]

    (tmpcv - tmpcv2).hist(bins=30)
    plt.title('test')
    (tcv - tcv2).hist(bins=10)
    plt.title('mineer')
    plt.show()

In [259]:
len(target_inner)

24

主要是看图，看日月的波动。

对于之前提交的几次里，交集共 24 个。 
其中找到2个波动大，应该不是挖矿的：
* 855996491, 2251440776

波动小的，应该是挖矿的 
* 179458306,179569820,1606708811,2071313507,2256064355,2347718610,2427050072
前边曲线波动小，最后一小段上升，应该是挖矿。
* 2186749200 

此外，并集中减去交集这部分剩余中
应该不是挖矿的可能是：
* 179433516 在35等大集合里没有，大概率没有，波动也大
* 179418058 波动大
* 362400993 挺奇怪的，波动也大
* 2576321385 月波动还好，日波动很大（这个日波动很奇怪）
* 887227796 日波动大

* 2717225084 较大
* 2471562086 月波动较大

波动小可能是挖矿的
* [179519109, 2115712219, 2245541933, 2268548573, 2271232929] 波动很小，可靠

日波动上看很小，可能也是挖矿的
* [2212416005, 2755128229, 2759317616] 日波动很小，或许吧

汇总：
可能是的：
* 179458306,179569820,1606708811,2071313507,2256064355,2347718610,2427050072, 2186749200, 179519109, 2115712219, 2245541933, 2268548573, 2271232929, 2212416005, 2755128229, 2759317616

可能不是的：
* 855996491, 2251440776, 179433516, 179418058, 362400993, 2576321385, 887227796, 2717225084, 2471562086


In [266]:
true = {179458306,179569820,1606708811,2071313507,2256064355,2347718610,2427050072, 2186749200, 179519109, 2115712219, 2245541933, 2268548573, 2271232929, 2212416005, 2755128229, 2759317616}
false = {855996491, 2251440776, 179433516, 179418058, 362400993, 2576321385, 887227796, 2717225084, 2471562086}

print(len(true), len(false))
infot(true)
print()
infot(false)
# 2115712219, 179519109, 2271232929, 2245541933

16 9
base_105_33.csv set() 16
base_37_27.csv {2115712219, 179519109} 14
base_182_34.csv set() 16
base_551_35.csv set() 16
base_68_29.csv {2271232929, 2245541933} 14
base_34_27.csv {2271232929, 179519109, 2755128229, 2245541933, 2759317616, 2115712219, 2268548573} 9
base_39_27.csv {2759317616, 2271232929, 2755128229} 13
base_35_24.csv {2271232929, 2245541933, 179519109, 2755128229} 12

base_105_33.csv {362400993, 179433516} 7
base_37_27.csv {362400993, 179418058, 179433516, 887227796} 5
base_182_34.csv {362400993, 179433516} 7
base_551_35.csv {362400993, 179433516} 7
base_68_29.csv {362400993, 2576321385, 179418058, 179433516, 887227796, 2717225084} 3
base_34_27.csv {2717225084} 8
base_39_27.csv {2576321385, 887227796} 7
base_35_24.csv {362400993, 179418058, 887227796} 6


In [269]:
# submit(false, 'submit_3_10_2.csv')
# info('submit_3_10_1.csv')

base_105_33.csv set() 16
base_37_27.csv {2115712219, 179519109} 14
base_182_34.csv set() 16
base_551_35.csv set() 16
base_68_29.csv {2271232929, 2245541933} 14
base_34_27.csv {2271232929, 2755128229, 179519109, 2245541933, 2759317616, 2115712219, 2268548573} 9
base_39_27.csv {2759317616, 2271232929, 2755128229} 13
base_35_24.csv {2271232929, 2245541933, 2755128229, 179519109} 12


# 发现了6个不是miner的id，去掉看看

In [316]:
not_miners = load_ids('submit_3_10_1.csv') - load_ids('submit_3_10_3.csv')

for i in Path('submit_csv').glob('3_10_base*.csv'):
    if len(load_ids(i.name)) > 50:
        continue
    print(i.name)

3_10_base_32_27.csv
3_10_base_34_27.csv
3_10_base_35_27.csv
3_10_base_32_24.csv


In [323]:
info('3_10_base_32_27.csv')

3_10_base_32_27.csv
3_10_base_98_33.csv set() 32
3_10_base_32_27.csv set() 32
3_10_base_175_34.csv set() 32
3_10_base_544_35.csv set() 32
3_10_base_63_29.csv {2576321385, 2717225084} 30
3_10_base_34_27.csv {2852503463, 2825175309, 2533183958, 2319973783, 2717225084} 27
3_10_base_35_27.csv {2576321385, 1916407803, 2817362052} 29
3_10_base_32_24.csv {2817362052, 2741872006, 2238809293, 2825175309, 2319973783} 27


* 3_10_base_32_27 3_10_base_63_29 30个一样。不知道会不会29个都在里边。
* 3_10_base_32_27 3_10_base_34_27 27个一样，不知道会不会这27个都是。

In [324]:
ids = load_ids('3_10_base_32_27.csv') &  load_ids('3_10_base_34_27.csv')
infot(ids)

3_10_base_98_33.csv set() 27
3_10_base_32_27.csv set() 27
3_10_base_175_34.csv set() 27
3_10_base_544_35.csv set() 27
3_10_base_63_29.csv {2576321385} 26
3_10_base_34_27.csv set() 27
3_10_base_35_27.csv {2576321385, 1916407803, 2817362052} 24
3_10_base_32_24.csv {2817362052, 2238809293, 2741872006} 24


In [325]:
ids = ids & load_ids('3_10_base_32_24.csv')
infot(ids)


3_10_base_98_33.csv set() 24
3_10_base_32_27.csv set() 24
3_10_base_175_34.csv set() 24
3_10_base_544_35.csv set() 24
3_10_base_63_29.csv {2576321385} 23
3_10_base_34_27.csv set() 24
3_10_base_35_27.csv {2576321385, 1916407803} 22
3_10_base_32_24.csv set() 24


In [329]:

diff('3_10_base_98_33.csv',('3_10_base_63_29.csv'))



3_10_base_98_33.csv 比 3_10_base_63_29.csv 0.505
多了:  44
少了:  9
都有:  54



({179408941,
  179409846,
  179416866,
  179418058,
  179458306,
  179466000,
  179474816,
  179507355,
  179508339,
  179509793,
  179516332,
  179546225,
  179558804,
  179562105,
  179569085,
  179569820,
  179571678,
  179574570,
  855996491,
  874996400,
  880030471,
  887227796,
  945474457,
  1429892107,
  1476442068,
  1531017123,
  1541520478,
  1566387227,
  1606708811,
  1862376457,
  1862415085,
  1868919231,
  1916407803,
  2044965214,
  2045098996,
  2071313507,
  2098712195,
  2103775652,
  2172970175,
  2173276768,
  2181712126,
  2186749200,
  2190112718,
  2194274407,
  2212416005,
  2212577893,
  2225355472,
  2238809293,
  2245473740,
  2245547066,
  2251440776,
  2256064355,
  2256067674,
  2271535997,
  2271557820,
  2273814498,
  2319973783,
  2326835483,
  2329827263,
  2347718608,
  2347718610,
  2372722956,
  2427050072,
  2442626095,
  2445049876,
  2451164292,
  2451165982,
  2471562086,
  2479963778,
  2486242927,
  2496032641,
  2523401557,
  2533183958,
 

In [277]:
load_ids('submit_3_10_1.csv') - load_ids('submit_3_10_3.csv')

{179519109,
 2115712219,
 2245541933,
 2268548573,
 2271232929,
 2755128229,
 2759317616}

In [258]:
print(len(target_all))
month = ds.month[ds.month.pq_z != 0]
day = ds.day[ds.day.kwh != 0]


# inner
# 855996491, 2251440776 波动大
# 179458306,179569820,1606708811,2071313507,2256064355,2347718610,2427050072 波动很小，可靠
# 2186749200 前边曲线波动小，最后一小段上升，比较可靠

# all
# 179433516 在35等大集合里没有，大概率没有，波动也大
# 179418058 波动大
# 362400993 挺奇怪的，波动也大
# 2576321385 月波动还好，日波动很大（这个日波动很奇怪）
# 887227796 日波动大

# 2717225084 较大
# 2471562086 月波动较大
# [179519109, 2115712219, 2245541933, 2268548573, 2271232929] 波动很小，可靠

# [2212416005, 2755128229, 2759317616] 日波动很小，或许吧
i = target_all
# i = target_inner
i = i - {179433516, 855996491, 2251440776} - {179458306,179569820,1606708811,2071313507,2256064355,2347718610,2427050072} - {2186749200} - {179418058, 362400993, 2576321385} - {2717225084, 887227796} - {179519109, 2115712219, 2245541933, 2268548573, 2271232929} - {2212416005, 2755128229, 2759317616}
infot(i)
print(len(i))
# cv, cvm = checkcv(i)
# i = cv[cv.pq_z < .15].index.values

# cv, cvm = checkcv(i, 'kwh')
# i = cv[cv.kwh < .1].index.values
for k in C.minerids:
    k = [k]
    plt_month(list(k),month, cols=['pq_z'])
    plt_day(list(k), day, cols=['kwh'])
print('asdfasfasdfasdf')

for k in i:
    k = [k]
    plt_month(list(k),month, cols=['pq_z'])
    plt_day(list(k), day, cols=['kwh', 'kwh_pap_r4'])

# # plt_month(list(i),month)
# # plt_month(list(i),month, cols=['pq_f', 'pq_p', 'pq_g', 'pq_z'])
# plt_month(list(i),month, cols=['pq_z'])

# # plt_month(C.minerids,month)
# # plt_month(C.minerids,month, cols=['pq_z'])


# # plt_day(list(i),day)
# plt_day(list(i), day, cols=['kwh'])
# # plt_day(list(i),day, cols=['kwh_pap_r2', 'kwh_pap_r3', 'kwh_pap_r4', 'kwh_pap_r1', 'kwh'])

# plt_day(C.minerids,day)
# plt_day(C.minerids,day, cols=['kwh'])
# plt_day(C.minerids,day, cols=['kwh_pap_r2', 'kwh_pap_r3', 'kwh_pap_r4', 'kwh_pap_r1', 'kwh'])

48
base_105_33.csv {1912367373} 23
base_37_27.csv {2745781539, 2602819207, 1429892107, 2624797677, 1912367373} 19
base_182_34.csv set() 24
base_551_35.csv set() 24
base_68_29.csv {1912367373} 23
base_34_27.csv {2852503463, 1429892107, 2825175309, 2624797677, 2533183958, 2319973783} 18
base_39_27.csv {2745781539, 2817362052, 1912367373, 1916407803} 20
base_35_24.csv {2817362052, 2741872006, 2238809293, 1912367373, 2825175309, 2319973783} 18
24
[329833811]


[329833811]


[2294741747]


[2294741747]


[2654985038]


...

[2717225077]


[1916407803]


[1916407803]


[2172970175]


[2172970175]
